In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
## Installer datamaestro et datamaestro-ml pip install datamaestro datamaestro-ml
import datamaestro
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.nn.functional import linear
from torch.nn import MSELoss
from tp1 import MSE, linear, Context

## Normalize boston Data

In [4]:
data=datamaestro.prepare_dataset("edu.uci.boston")
colnames, datax, datay = data.data()
datax = (datax-datax.min(axis=0))/(datax.max(axis=0)-datax.min(axis=0))

## Apply TME1 functions

In [6]:
#writer = SummaryWriter()

# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Les paramètres du modèle à optimiser
w = torch.randn(13, 1)
b = torch.randn(1)

epsilon = 0.001

writer = SummaryWriter()
for n_iter in range(1000):
    # Pass Forward
    ctx1 = Context()
    ctx2 = Context()

    yhat = linear.forward(ctx1,x,w,b)
    loss = MSE.forward(ctx2,yhat,y)

    # Back propagation
    grad_yhat, grad_y = MSE.backward(ctx2, 1)
    grad_x, grad_w, grad_b = linear.backward(ctx1, grad_yhat)

    #import ipdb;ipdb.set_trace()

    # Tensorboard visualization
    # To visualize type command : tensorboard --logdir=runs in current directory
    writer.add_scalar('Loss/train', loss, n_iter)
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

    # Updating parameters
    w -= epsilon*grad_w
    b -= epsilon*grad_b

#writer.close()

Itérations 0: loss 712.0059814453125
Itérations 100: loss 254.4786376953125
Itérations 200: loss 146.6306610107422
Itérations 300: loss 116.59663391113281
Itérations 400: loss 104.45825958251953
Itérations 500: loss 96.93397521972656
Itérations 600: loss 91.01441955566406
Itérations 700: loss 85.96133422851562
Itérations 800: loss 81.54600524902344
Itérations 900: loss 77.66048431396484


## Apply with torch functions

In [13]:

# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Les paramètres du modèle à optimiser
w =  torch.nn.Parameter(torch.randn((1, 13)))
b =  torch.nn.Parameter(torch.randn(1))

epsilon = 0.001

optim = torch.optim.SGD(params=[w,b],lr=epsilon)
optim.zero_grad()

mse = MSELoss()

size_batch = 100

writer = SummaryWriter()
for n_iter in range(1000):
    
    indices = torch.randperm(len(x))
    for i in range(int(np.ceil((len(x)/size_batch)))):
        # Pass Forward
        index = indices[i*size_batch:(i+1)*size_batch]
        x_batch = x[index]
        y_batch = y[index]
        yhat = torch.nn.functional.linear(x_batch,w,b)
        loss = mse(yhat,y_batch)

        loss.backward()

        #import ipdb;ipdb.set_trace()

        # Tensorboard visualization
        # To visualize type command : tensorboard --logdir=runs in current directory
        writer.add_scalar('Loss/train', loss, n_iter)
        
        # Updating parameters & reset grad
        optim.step()
        optim.zero_grad()
        
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

writer.close()

Itérations 0: loss 253.6772918701172
Itérations 100: loss 99.35590362548828
Itérations 200: loss 8.592985153198242
Itérations 300: loss 133.64051818847656
Itérations 400: loss 74.8947525024414
Itérations 500: loss 15.232146263122559
Itérations 600: loss 18.405847549438477
Itérations 700: loss 19.734079360961914
Itérations 800: loss 45.11915969848633
Itérations 900: loss 17.467031478881836


## Try 2 couches (linear->tanh->linear->MSE)

In [33]:
# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Les paramètres du modèle à optimiser
w1 =  torch.nn.Parameter(torch.randn((1, 13)))
b1 =  torch.nn.Parameter(torch.randn(1))

w2 =  torch.nn.Parameter(torch.randn((1,1)))
b2 =  torch.nn.Parameter(torch.randn(1))

epsilon = 0.001

optim = torch.optim.SGD(params=[w1,b1,w2,b2],lr=epsilon)
optim.zero_grad()

mse = MSELoss()

size_batch = len(x)

#writer = SummaryWriter()
for n_iter in range(1000):
    
    indices = torch.randperm(len(x))
    for i in range(int(np.ceil((len(x)/size_batch)))):
        #Compute sample
        index = indices[i*size_batch:(i+1)*size_batch]
        x_batch = x[index]
        y_batch = y[index]
        
        # Pass Forward
        out = torch.nn.functional.linear(x_batch,w1,b1)
        out = torch.nn.functional.tanh(out)
        yhat = torch.nn.functional.linear(out,w2,b2)
        loss = mse(yhat,y_batch)
        
        # Pass backward
        loss.backward()
        
        # Updating parameters & reset grad
        optim.step()
        optim.zero_grad()
        
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

#writer.close()

Itérations 0: loss 594.3727416992188
Itérations 100: loss 314.0970458984375
Itérations 200: loss 187.53256225585938
Itérations 300: loss 130.70249938964844
Itérations 400: loss 105.19783782958984
Itérations 500: loss 93.75227355957031
Itérations 600: loss 88.61532592773438
Itérations 700: loss 86.30905151367188
Itérations 800: loss 85.27283477783203
Itérations 900: loss 84.8064193725586


## Use Module

In [35]:
# Les données supervisées
x = torch.tensor(datax).float()
y = torch.tensor(datay.reshape((len(datay), 1))).float()

# Module
model = torch.nn.Sequential(
          torch.nn.Linear(13, 1, bias=True),
          torch.nn.Tanh(),
          torch.nn.Linear(1, 1, bias=True)
        )

epsilon = 0.001

optim = torch.optim.SGD(params=model.parameters(),lr=epsilon)
optim.zero_grad()

criterion = MSELoss()

size_batch = len(x)

#writer = SummaryWriter()
for n_iter in range(1000):
    
    indices = torch.randperm(len(x))
    for i in range(int(np.ceil((len(x)/size_batch)))):
        #Compute sample
        index = indices[i*size_batch:(i+1)*size_batch]
        x_batch = x[index]
        y_batch = y[index]
        
        # Pass Forward
        yhat = model(x_batch)
        loss = criterion(yhat, y_batch)
        
        # Pass backward
        loss.backward()
        
        # Updating parameters & reset grad
        optim.step()
        optim.zero_grad()
        
    if(n_iter%100==0):
        print(f"Itérations {n_iter}: loss {loss}")

#writer.close()

Itérations 0: loss 627.5291137695312
Itérations 100: loss 333.9671325683594
Itérations 200: loss 196.41893005371094
Itérations 300: loss 134.69065856933594
Itérations 400: loss 106.9928207397461
Itérations 500: loss 94.56427764892578
Itérations 600: loss 88.98666381835938
Itérations 700: loss 86.48289489746094
Itérations 800: loss 85.35829162597656
Itérations 900: loss 84.85257720947266
